# Formal Methods of Ranking Alternative Choices and Applications to Architecture Selection

### Dr. Jon Fox jon.fox@gtri.gatech.edu
### Principal Research Scientist, GTRI ASL

---

# Methods of Multi-Criteria Decision Making

1. Analytical Hierarchy Method
2. Analytical Network Method
3. Fuzzy Analytical Hierarchy Method
4. Technique for Order of Preference by Similarity to Ideal Solution (TOPSIS) 
5. Fuzzy TOPSIS

---

# Introduction

* You need to make a decision.
* You need to make a decision in a rational way. 
* You need to standardize and document your decision making process.
* You can break apart the criteria for your decision and assign importance to different potential benefits and costs. 
* You can assign scores how alternative choices rank in the criteria for your decision.

--- 

# Analytical Hierarchy Process (AHP)

"In using the AHP to model a problem, one needs a hierarchic or a network
structure to represent that problem, as well as pairwise comparisons to establish
relations within the structure."

* Invented and refined by Thomas L. Saaty (1926-2017)
1. Select a Goal
2. Select criteria for reaching the goal, such as expected qualities attributes. 
3. Determine the relative importance of different attributes or critera with respect to the goal. 
4. Convert that set of pairwise factors into a matrix. Normalize it. Calculate eigenvectors. 



## Buy a phone

![Example 1](example1.svg)

## Example of Klaus Geopel

https://www.youtube.com/watch?v=18GWVtVAAzs 

* Memory is moderately more important than color. ( color/memory = $1/3$)
* Availability is somewhat more important than color (color/availability = $1/2$ )
* Memory and Availability of equal importance (memory/availabilty = 1 )

|               | Color  | Memory | Availability |
|---------------|--------|--------|---------------|
| Color         |   1    |    1/3 |  1/2          |
| Memory        |   3    |    1   |  1            |
| Availability  |   2    |    1   |  1            |



## Compute the eigenvector of the normalized pairwise-comparison matrix. 

### Sum the columns of the $n \times n$ matrix



In [1]:
# import numpy as np
import array as ar

In [5]:
#pairwise = ar.array( 'd', [ [1, 1/3.0 , 1/2.0], [3, 1, 1],[2, 1, 1] ])
pairwise = [ [1, 1/3.0 , 1/2.0], [3, 1, 1],[2, 1, 1] ]

In [11]:
def colsum( arr, i):
    s = 0
    for row in arr:
        s += row[i]
    return s
        
colsum(pairwise, 1)

2.333333333333333

In [14]:
def shape(arr):
    a = len(arr)
    b = len(arr[0])
    return[a,b]

shape(pairwise)

[3, 3]

In [17]:
def zeros(shape):
    arr = []
    for i in range(shape[0]):
        sub_arr = []
        for j in range(shape[1]):
            sub_arr.append( 0.0 )
        arr.append(sub_arr)
    return arr

zeros( [3,2])

[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]

### Compute sum of columns of pair-wise comparison matrix

In [15]:
def calc_colsum( pairwise):
    # colsum = lambda arr,i: sum(arr[:, i])
    cols = []
    for a in range(shape(pairwise)[1]):
        x = colsum( pairwise, a)
        cols.append(x)
    return cols

colsums = calc_colsum( pairwise)
colsums

[6, 2.333333333333333, 2.5]

### Normalize the array columns to the column sum of the pairwise comparison matrix

In [39]:
def calc_normalized_pairwise( pairwise):
    cols = calc_colsum(pairwise)
    pairwise_normalized = zeros(shape(pairwise))
    for i in range(shape(pairwise)[1]):
        for j in range(shape(pairwise)[0]):
            pairwise_normalized[j][i] = pairwise[j][i]/cols[i]
    return pairwise_normalized

cnp = calc_normalized_pairwise( pairwise)
cnp

[[0.16666666666666666, 0.14285714285714288, 0.2],
 [0.5, 0.4285714285714286, 0.4],
 [0.3333333333333333, 0.4285714285714286, 0.4]]

In [103]:
import numpy.linalg

w, v = numpy.linalg.eig(cnp)

array([ 1.        +0.j        , -0.00238095+0.06896547j,
       -0.00238095-0.06896547j])

In [109]:
np.real(v[:,0])/2

array([0.13886475, 0.35101923, 0.3278751 ])

### Compute the Criteria Weights by averaging the weights along the rows of the normalized pairwise comparison matrix

$$ x = \left ( \begin{array}{c}
                   \frac {\sum \rm{row_1}}{n} \\
                   \frac {\sum \rm{row_2}}{n} \\
                   \frac {\sum \rm{row_2}}{n} 
               \end{array}
       \right ) $$

In [54]:
def calc_crit_weights( normalized):
    crit_weights = []
    width = shape(normalized)[0]
    for j in range(width):
        c = 0
        for i in range(shape(normalized)[1]):
            c += normalized[j][i]/width
        crit_weights.append(c)
    return crit_weights
cw = calc_crit_weights( cnp)
cw
cw_col = [ [i] for i in cw]
cw_col

[[0.16984126984126985], [0.44285714285714284], [0.38730158730158726]]

In [49]:
# https://stackoverflow.com/questions/10508021/matrix-multiplication-in-pure-python

def matmult(a,b):
    "multiples two 2-dimensional matrices together"
    zip_b = zip(*b)
    # uncomment next line if python 3 : 
    zip_b = list(zip_b)
    return [[sum(ele_a*ele_b for ele_a, ele_b in zip(row_a, col_b)) 
             for col_b in zip_b] for row_a in a]

x = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
y = [[1.0,2.0],[1,2],[3,4]]
matmult(x,y)

[[12.0, 18.0], [27.0, 42.0], [42.0, 66.0], [57.0, 90.0]]

In [50]:
list(zip(*y))

[(1.0, 1, 3), (2.0, 2, 4)]

In [55]:
#cw = np.array(cw)
#res = np.matmul( cnp, cw)
#res, cw/res

res = matmult(cnp, cw_col)
res

[[0.16903250188964475], [0.42963718820861674], [0.40133030990173846]]

In [59]:
[ f/res[i][0] for i,f in enumerate(cw) ]

[1.0047846889952154, 1.0307700427508313, 0.9650444477926774]

## Iterate to get a better eigenvector calculation

In [137]:
cw2 = calc_crit_weights( np.matmul( cnp, cnp))
cw2, sum(cw2)

([0.16903250188964478, 0.42963718820861685, 0.4013303099017385],
 1.0000000000000002)

In [138]:
cw = np.array(cw2)
res = np.matmul( cnp, cw)
res

array([0.16981489, 0.4291786 , 0.40100651])

In [139]:
cw/res

array([0.99539272, 1.00106853, 1.00080746])

In [140]:
crit_weights = np.array( crit_weights)
crit_mult = np.stack ( width*[ crit_weights],0)
crit_mult

array([[0.16984127, 0.44285714, 0.38730159],
       [0.16984127, 0.44285714, 0.38730159],
       [0.16984127, 0.44285714, 0.38730159]])

## Calculate pairwise decision matrix for sub-criteria

|        | Pink | Blue | Green | Black | Red   |
|--------|------|------|-------|-------|-------|
| Pink   |  1   |   2  |   3   |   1/2 |  1/4  |
| Blue   | 1/2  |  1   |   3   |  1/4  |  1/3  |
| Green  | 1/3  |  1/3 |  1    |  1/7  |  1/7  |
| Black  |  2   |  4   |   7   |   1   |  1/3  |
| Red    |  4   |  3   |   7   |   3   |   1   |




## Compute the normalized principal eigenvector of the matrix

| Color  |  Weight |
|--------|---------|
| Pink   |   0.13  |
| Blue   |   0.12  |
| Green  |   0.05  |
| Black  |   0.22  |
| Red    |   0.49  |

---

## We can say that the sub-criteria should be weighted overall by the color weight

$$ w_{pink} \times w_{color} = 0.13 \times 0.17 = 0.022 $$


| Color  |  Weight | Over-all weight |
|--------|---------|-----------------|
| Pink   |   0.13  | 0.022           |
| Blue   |   0.12  | 0.02            |
| Green  |   0.05  | 0.009           |
| Black  |   0.22  | 0.037           |
| Red    |   0.49  | 0.083           |

| Memory  | Weight  | Over-all weight |
|---------|---------|-----------------|
| 8 MB    | 0.06    | 0.02            |
| 16 MB   | 0.07    | 0.03            |
| 32 MB   | 0.43    | 0.19            |
| 64 MB   | 0.44    | 0.19            |

| Delivery | Weight |  Over-all weight |
|----------|--------|------------------|
| Immediate | 0.46   | 0.18            |
| One Week. | 0.43.  | 0.17            |
| 4 weeks.  | 0.12   | 0.05            |

---

### Examine the Alternatives 

| Model   | Attributes                         |
|---------|------------------------------------|
| Model 1 | Pink, 32 MB, immediately available | 
| Model 2 | Blue, 16 MB, immediately available |
| Model 3 | Black, 32 MB, available in 1 week  |
| Model 4 | Red, 64 MB, available in 4 weeks   |

* Each of the alternatives is a vector of which we can calculate a score. 

### Examine the Alternatives 

| Model   | Attributes                         | Score |
|---------|------------------------------------|-------|
| Model 1 | Pink, 32 MB, immediately available | 0.39  |
| Model 2 | Blue, 16 MB, immediately available | 0.23  |
| Model 3 | Black, 32 MB, available in 1 week  | 0.40  |
| Model 4 | Red, 64 MB, available in 4 weeks   |0.32   |

